In [1]:
import io
import os
import re
import sys
import csv
import gzip
import glob
import time 
import json
import math
import pickle
import secrets
import multiprocessing
import subprocess
import resource
import pyreadr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import statsmodels.api as sm
import random
import itertools
import collections
import pyreadr
import pywfa
from collections import Counter
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import normalize
from sklearn.preprocessing import minmax_scale

import multiprocessing as mp
from functools import partial
from itertools import combinations_with_replacement

home_dir = os.environ.get("home_dir")
sys.path.append(f'{home_dir}software/lcwgsus/')
import lcwgsus
from lcwgsus.variables import *
from warnings import simplefilter
from hla_phase_functions import *
from hla_align_functions import *

from scipy.special import logsumexp
from scipy.stats import pearsonr
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

pd.options.mode.chained_assignment = None
%cd /well/band/users/rbx225/GAMCC

/gpfs3/well/band/users/rbx225/GAMCC


<Figure size 100x100 with 0 Axes>

# Alignment

## Optimising hyperparameters

In [16]:
score_diff_in_alignment_genes_ary = [0, 8]
n_mismatches_ary = [3, 5]
weight_ary = [True, False]

d = score_diff_in_alignment_genes_ary[0]
n = n_mismatches_ary[0]
w = weight_ary[0]

g = 'A'
s = 'IDT0481'

bam = f"/well/band/users/rbx225/GAMCC/data/bams/{s}.bam"
db_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/v3390_merged_only/'
as_dir = 'results/hla/imputation/WFA_alignments/v3390/'
hla_gene_information = pd.read_csv(HLA_GENE_INFORMATION_FILE, sep = ' ')

r1, _, mate, pair = hla_aligner(g, bam, db_dir, as_dir, hla_gene_information,
                               n_mismatches = n, score_diff_in_alignment_genes = d)

qmat = pyreadr.read_r(f'results/hla/imputation/QUILT_HLA_result_method/{s}/{g}/extracted.hla{g}.RData')['quilt']

a1 = set(pair.columns)
a2 = set(qmat.columns)

cols = sorted(list(a1.intersection(a2)))
pair = pair.loc[cols, cols]
qmat = qmat.loc[cols, cols]

pair = pair - pair.max().max()
mask = np.tril(np.ones(pair.shape), k=-1).astype(bool)
pair = pair.where(~mask, other=0)
pair = np.exp(pair)
pair = pair/pair.sum().sum()

if w:
    w1 = pair.max().max()
    w2 = qmat.max().max()
    w1 = w1/(w1+w2)
    w2 = 1-w1
    
    combined = (pair**w1)*(qmat**w2)
else:
    combined = pair*qmat
    
combined = combined/combined.sum().sum()
bestalleles = get_best_alleles(np.log(combined))

bestalleles.columns = ['bestallele1', 'bestallele2', 'post_prob', 'sums']
bestalleles['sample_number'] = 1
bestalleles['sample_name'] = sample
bestalleles = bestalleles[['sample_number', 'sample_name', 'bestallele1', 'bestallele2', 'post_prob', 'sums']]

bestalleles.to_csv(f'{reads_df_outdir}/{sample}/{gene}/quilt.hla.output.onlyreads.all.txt', index = False, header = True, sep = '\t')
bestalleles.iloc[[0], :].to_csv(f'{reads_df_outdir}/{sample}/{gene}/quilt.hla.output.onlyreads.topresult.txt', index = False, header = True, sep = '\t')

,allele1,allele2,prob,cum
0,A*23:01,A*23:01,0.999968,0.999968


## Comparing with QUILT

In [94]:
gene = 'B'
hla_gene_information = pd.read_csv('/well/band/users/rbx225/software/QUILT_sus/hla_ancillary_files/hla_gene_information.tsv', sep = ' ')
bam = "/well/band/users/rbx225/GAMCC/data/bams/IDT0561.bam"
db_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/v3390_oneKG_only/'
reads_apart_max=1000
reads_extend_max=1000
n_mismatches=5
assumed_bq=0.001
score_diff_in_alignment_genes = 0
n_unique_onefield = 100
phase=False

reads1 = get_chr6_reads(gene, bam, hla_gene_information, strict = True,
                            reads_apart_max = reads_apart_max, 
                            reads_extend_max = reads_extend_max)
    
rl = reads1['sequence'].str.len().mode().values[0]

columns = []
for g in HLA_GENES_ALL:
    db = pd.read_csv(f'{db_dir}{g}.ssv', sep = ' ')
    columns = columns + db.columns.tolist()
columns = np.array(columns)
columns_to_keep = np.where(np.char.startswith(columns, gene + '*'))[0]
target_alleles = columns[columns_to_keep]
sample = bam.split('/')[-1].split('.')[0]

likemat1 = pd.read_csv(f"/well/band/users/rbx225/GAMCC/results/hla/imputation/WFA_alignments/v3390_oneKG/{sample}/{gene}/AS_matrix.ssv", sep = ' ', index_col = 0).values
min_valid_prob = n_mismatches*np.log(assumed_bq) + (rl - n_mismatches)*np.log(1 - assumed_bq)

valid_indices1 = np.any(likemat1 >= min_valid_prob, axis=1)
likemat1, reads1 = likemat1[valid_indices1], reads1[valid_indices1]
likemat_all = likemat1
id1 = reads1.iloc[:, 0].to_numpy()

unique_ids = np.unique(id1)
likemat_mate = -600*np.ones((len(unique_ids), likemat_all.shape[1]))

for i, uid in enumerate(unique_ids):
    tmp = likemat_all[id1 == uid, :]

    keep = False
    unique = True
    for j in range(tmp.shape[0]):
        best_indices = np.where(tmp[j,:] == tmp[j,:].max())[0]
        best_alleles = columns[best_indices]
        aligned_genes = np.unique(np.array([s.split('*')[0] for s in best_alleles]))
        keep = keep or ((len(aligned_genes) == 1) and (aligned_genes[0] == gene))
        unique = unique and (gene in aligned_genes)

    if (keep and unique):
        likemat_mate[i, :] = np.sum(tmp, axis=0)

# Filtering out based on alignment scores

scoring_df = pd.DataFrame({'ID': unique_ids, 
                    'Target': likemat_mate[:,columns_to_keep].max(axis = 1), 
                    'Others': likemat_mate[:,~np.where(np.char.startswith(columns, gene + '*'))[0]].max(axis = 1)
                   })
scoring_df['diff'] = scoring_df['Target'] - scoring_df['Others']
scoring_df
valid_mask = ((scoring_df['diff'] > score_diff_in_alignment_genes) & (scoring_df['Target'] >= min_valid_prob)).tolist()

likemat_mate = likemat_mate[valid_mask, :][:, columns_to_keep]
unique_ids = unique_ids[valid_mask]
reads1 = reads1[reads1['ID'].isin(unique_ids)]
likemat_mate_df = pd.DataFrame(likemat_mate, index = unique_ids, columns=target_alleles)

df = likemat_mate_df.copy()
best_alignments = df.max(axis=1)
matching_cols = df.eq(best_alignments, axis=0).apply(lambda row: likemat_mate_df.columns[row].tolist(), axis=1)
one_field = matching_cols.apply(lambda cols: list(set(col.split(':')[0] for col in cols)))
counts = one_field.apply(len)
rows_to_remove = counts[counts >= n_unique_onefield].index
rows_to_keep = df.index.difference(rows_to_remove)

likemat_mate_df = likemat_mate_df.loc[rows_to_keep]
reads1 = reads1[reads1['ID'].isin(rows_to_keep)]
likemat_mate = likemat_mate_df.values

if phase:
    if likemat_mate.shape[0] > 2:
        likemat_mate_normalised = normalize(np.exp(likemat_mate - likemat_mate.max(axis = 1, keepdims = True)), axis=1, norm='l1')
        gmm = GaussianMixture(n_components=2, tol = 1e-10, max_iter = 1000, n_init = 50)
        gmm.fit(likemat_mate_normalised)
        labels = gmm.predict(likemat_mate_normalised)

        group1 = np.where(labels == 0)[0]
        group2 = np.where(labels == 1)[0]

        l1 = likemat_mate[group1, :].mean(axis=0)
        l2 = likemat_mate[group2, :].mean(axis=0)

        likemat_mate = np.vstack([l1, l2])

likemat_norm=likemat_mate-likemat_mate.max(axis = 1, keepdims = True)
likemat_norm=0.5*np.exp(likemat_norm)+1e-100

likemat_pair=pd.DataFrame(0, index=target_alleles, columns=target_alleles)
qq=likemat_pair*0
for i in range(likemat_mate.shape[0]):
    qq=qq*0
    m1=qq+likemat_norm[i,:]
    m1=m1+m1.T
    likemat_pair=likemat_pair+np.log(m1)

likemat_paired_df = pd.DataFrame(likemat_pair, index=target_alleles, columns=target_alleles)
lcwgsus.extract_unique_twofield(find_best_pair_alignment(likemat_paired_df)[0].unique())

array(['B*08:01', 'B*35:12'], dtype='<U7')

In [92]:
for i in range(likemat_mate.shape[0]):
    best = target_alleles[np.where(likemat_mate[i,:] == likemat_mate[i,:].max())[0]]
    print(lcwgsus.extract_unique_twofield(best))

['B*07:02' 'B*08:01' 'B*40:01' 'B*40:03' 'B*41:01' 'B*41:02' 'B*41:03'
 'B*42:01' 'B*42:02' 'B*42:08' 'B*44:03']
['B*35:01' 'B*35:02' 'B*35:03' 'B*35:04' 'B*35:05' 'B*35:06' 'B*35:08'
 'B*35:09' 'B*35:10' 'B*35:11' 'B*35:12' 'B*35:14' 'B*35:17' 'B*35:20'
 'B*35:22' 'B*35:23' 'B*35:40N' 'B*35:41' 'B*35:43' 'B*35:57' 'B*51:01'
 'B*51:02' 'B*51:06' 'B*51:07' 'B*51:08' 'B*51:09' 'B*51:83' 'B*52:01'
 'B*52:02' 'B*53:01' 'B*58:01' 'B*58:02' 'B*58:11' 'B*78:01' 'B*78:04']
['B*08:01']
['B*35:01' 'B*35:02' 'B*35:03' 'B*35:05' 'B*35:12' 'B*35:14' 'B*35:17'
 'B*58:02']
['B*07:02' 'B*08:01' 'B*27:07' 'B*39:01' 'B*40:02' 'B*40:03' 'B*40:04'
 'B*40:05' 'B*40:06' 'B*41:01' 'B*41:02' 'B*42:01' 'B*42:02' 'B*48:01']


## BWA alignment

In [38]:
def pos_to_HLA_allele(r, index_df):
    gene, pos = r[['gene', 'pos']]
    tmp = index_df[index_df['gene'] == gene]
    tmp = tmp[(tmp['Start'] <= pos) & (tmp['End'] >= pos)]
    if len(tmp) == 0:
        r['pos'] = 'N/A'
    else:
        r['pos'] = tmp['Allele'].values[0]
    
    return r

def map_alleles_to_array(target_alleles, oneread, misalignment_score = -151):
    allele_to_as = dict(zip(oneread['pos'], oneread['AS']))
    result = np.array([allele_to_as.get(a, misalignment_score) for a in target_alleles])
    return result

def get_as_from_one_read_pair(rid, onepair, gene, index_df, target_alleles):
    scores_ary_list = []

    unique = False
    aligned = True

    for i in [1,2]:
        oneread = onepair[onepair['pair'] == i]
        if len(oneread) == 0:
            pass
        else:
            aligned_genes = oneread['gene'].unique()
            if gene in aligned_genes:
                target_max = oneread[oneread['gene'] == gene]['AS'].max()
            else:
                target_max = 0

            others = oneread[oneread['gene'] != gene]
            if len(others) != 0:
                others_max = others['AS'].max()
            else:
                others_max = 0

            unique = unique or ((len(aligned_genes) == 1) and (aligned_genes[0] == gene))
            aligned = aligned and (gene in alinged_genes)

            oneread = oneread[oneread['gene'] == gene]
            oneread = oneread.apply(pos_to_HLA_allele, args = (index_df,), axis = 1)
            scores_ary = map_alleles_to_array(target_alleles, oneread)
            scores_ary_list.append(scores_ary)

    if (unique and aligned):
        final_scores_ary = np.zeros(len(scores_ary_list)[0])
        for s_ary in scores_ary_list:
            final_scores_ary = final_scores_ary + s_ary
        return rid, final_scores_ary
    else:
        return '', None

In [28]:
n_mismatch = 5
min_valid_score = 151 - 5*n_mismatch
gene = 'B'

index_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/fasta/v3390_oneKG/'
files = glob.glob(f'{index_dir}*.index.tsv')
index_df = []
for f in files:
    tmp = pd.read_csv(f, sep = '\t')
    tmp['gene'] = tmp['Allele'].str.split('*').str.get(0)
    index_df.append(tmp)
index_df = pd.concat(index_df).reset_index(drop = True)
target_alleles = index_df[index_df['gene'] == gene]['Allele'].to_numpy()

bam = 'data/realigned_bams/v3390_oneKG/IDT0717.bam'
command = f"samtools view {bam}"
reads = subprocess.run(command, shell = True, capture_output = True, text = True).stdout[:-1].split('\n')

reads = [i.split('\t') for i in reads if '##' not in i]
reads = pd.DataFrame(reads)
reads.columns = [
    "ID", "flag", "chr", "pos", "map_quality", "CIGAR", "chr_alt", "pos_alt", "insert_size", "sequence", "base_quality"
] + [f'col{i}' for i in range(11, reads.shape[1])]

reads['pos'] = reads['pos'].astype(int)
reads['pos_alt'] = reads['pos_alt'].astype(int)

reads = reads[reads['CIGAR'] != '*']
reads['NM'] = reads['col11'].str.split(':').str.get(2).astype(int)
reads['AS'] = reads['col13'].str.split(':').str.get(2).astype(int)
reads['gene'] = reads['chr'].str.split('-').str.get(1)
reads['pair'] = reads['ID'].str.split('_').str.get(1).astype(int)
reads['ID'] = reads['ID'].str.split('_').str.get(0)
reads = reads[['ID', 'pair', 'gene', 'pos', 'AS', 'NM']]
reads

KeyboardInterrupt: 

In [56]:
reads

,ID,pair,gene,pos,AS,NM
0,A00711:753:HHCG3DSX7:4:1115:13241:11898,2,A,11,90,5
1,A00711:753:HHCG3DSX7:4:1475:4924:10989,2,A,11,90,5
2,A00711:753:HHCG3DSX7:4:1573:2582:37027,2,A,11,79,0
3,A00711:753:HHCG3DSX7:4:2242:7808:4586,2,A,11,80,4
4,A00711:748:HHFNVDSX7:4:1602:26603:32847,1,A,13,55,0
...,...,...,...,...,...,...
2248794,A00711:748:HHFNVDSX7:4:2419:18285:1251,2,Y,7788,151,0
2248795,A00711:753:HHCG3DSX7:4:2649:24659:5776,1,Y,7810,151,0
2248796,A00711:753:HHCG3DSX7:4:1418:29875:36229,1,Y,7867,151,0
2248797,A00711:753:HHCG3DSX7:4:1418:29875:36229,2,Y,7970,151,0


In [58]:
onepair[onepair['pair'] == 1]

,ID,pair,gene,pos,AS,NM
15603,A00711:753:HHCG3DSX7:4:2354:20817:19601,1,B,1293394,151,0
16083,A00711:753:HHCG3DSX7:4:2354:20817:19601,1,B,1622564,151,0
16090,A00711:753:HHCG3DSX7:4:2354:20817:19601,1,B,1626189,151,0
16092,A00711:753:HHCG3DSX7:4:2354:20817:19601,1,B,1629661,151,0
16098,A00711:753:HHCG3DSX7:4:2354:20817:19601,1,B,1633683,151,0
...,...,...,...,...,...,...
23528,A00711:753:HHCG3DSX7:4:2354:20817:19601,1,B,4874670,151,0
23535,A00711:753:HHCG3DSX7:4:2354:20817:19601,1,B,4877695,151,0
23543,A00711:753:HHCG3DSX7:4:2354:20817:19601,1,B,4880759,151,0
23555,A00711:753:HHCG3DSX7:4:2354:20817:19601,1,B,4884386,151,0


In [55]:
rid = 'A00711:753:HHCG3DSX7:4:2354:20817:19601'
onepair = reads[reads['ID'] == rid]
gene = 'B'
index_df = index_df
target_alleles = target_alleles

scores_ary_list = []

unique = False
aligned = True

for i in [1,2]:
    oneread = onepair[onepair['pair'] == i]
    if len(oneread) == 0:
        pass
    else:
        aligned_genes = oneread['gene'].unique()
        if gene in aligned_genes:
            target_max = oneread[oneread['gene'] == gene]['AS'].max()
        else:
            target_max = 0

        others = oneread[oneread['gene'] != gene]
        if len(others) != 0:
            others_max = others['AS'].max()
        else:
            others_max = 0

        unique = unique or ((len(aligned_genes) == 1) and (aligned_genes[0] == gene))
        aligned = aligned and (gene in alinged_genes)

        oneread = oneread[oneread['gene'] == gene]
        oneread = oneread.apply(pos_to_HLA_allele, args = (index_df,), axis = 1)
        scores_ary = map_alleles_to_array(target_alleles, oneread)
        scores_ary_list.append(scores_ary)
scores_ary_list
# if (unique and aligned):
#     final_scores_ary = np.zeros(len(scores_ary_list)[0])
#     for s_ary in scores_ary_list:
#         final_scores_ary = final_scores_ary + s_ary
#     return rid, final_scores_ary

NameError: name 'alinged_genes' is not defined

In [45]:
unique_ids = []

rid = 'A00711:753:HHCG3DSX7:4:2354:20817:19601'
onepair = reads[reads['ID'] == rid]
gene = 'B'
index_df = index_df
target_alleles = target_alleles


    
rid, final_scores_ary = get_as_from_one_read_pair(rid, onepair, gene, index_df, target_alleles)
rid

''

## Examining Individual cases

### IDT0486-A

In [2]:
gene = 'A'
bam = "/well/band/users/rbx225/GAMCC/data/bams/IDT0606.bam"
db_dir = '/well/band/users/rbx225/recyclable_files/hla_reference_files/v3390_merged_only/'
as_dir = 'results/hla/imputation/WFA_alignments/v3390/'
hla_gene_information = pd.read_csv(HLA_GENE_INFORMATION_FILE, sep = ' ')

r1, _, mate, pair = hla_aligner(gene, bam, db_dir, as_dir, hla_gene_information)

,A*01:01,A*01:02,A*02:01,A*02:02,A*02:03,A*02:04,A*02:05,A*02:06,A*02:07,A*02:08,...,A*68:01,A*68:02,A*68:03,A*68:15,A*68:17,A*69:01,A*74:01,A*74:02,A*74:03,A*80:01
A*01:01,-141.99005,-149.331113,-124.092316,-136.088788,-132.090182,-132.090182,-136.089118,-132.108015,-132.090182,-136.089124,...,-128.109415,-132.108351,-124.110150,-132.108351,-128.109415,-124.110480,-136.764101,-136.764056,-140.088060,-144.780796
A*01:02,0.00000,-191.478938,-128.109421,-148.085931,-140.105892,-140.105892,-151.409889,-144.086995,-140.105892,-152.084867,...,-143.412017,-148.085931,-136.089125,-148.085931,-143.412017,-139.413082,-150.779345,-146.780409,-158.777209,-164.082340
A*02:01,0.00000,0.000000,-202.698012,-201.390380,-209.291757,-209.291757,-205.985968,-209.292087,-209.291757,-205.985968,...,-208.617115,-209.292093,-204.694075,-209.292093,-208.617115,-205.293158,-204.084308,-208.065409,-208.029072,-156.744182
A*02:02,0.00000,0.000000,0.000000,-219.996677,-213.351191,-213.351191,-220.671655,-216.675149,-211.386659,-220.689488,...,-213.369024,-217.367960,-208.695442,-217.367960,-213.369024,-209.370419,-212.757442,-214.771005,-218.769658,-172.721756
A*02:03,0.00000,0.000000,0.000000,0.000000,-219.920775,-219.920775,-217.964606,-218.658090,-214.674670,-217.964936,...,-217.289628,-220.613916,-213.290693,-220.613916,-217.289628,-213.308526,-208.776054,-212.757111,-216.026944,-168.065675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A*69:01,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-219.920777,-200.778520,-204.759622,-208.047236,-168.065677
A*74:01,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-259.959064,-256.652940,-261.309021,-168.759159
A*74:02,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-259.959063,-259.940558,-168.759115
A*74:03,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-269.894599,-172.758090


In [25]:
a1 = 'A*01:01'
a2 = 'A*36:01'

print(x.loc[a1, a2])
print(y.loc[a1, a2])
print(z.loc[a1, a2])

0.04651972753484129
0.22923436030194602
0.6840473683209383


In [26]:
a1 = 'A*01:01'
a2 = 'A*01:01'

print(x.loc[a1, a2])
print(y.loc[a1, a2])
print(z.loc[a1, a2])

0.0011079174528854001
0.450213985456336
0.1936214175202403


In [27]:
data = pyreadr.read_r('results/hla/imputation/QUILT_HLA_result_method/IDT0606/A/test.RData')
x,y,z = data['x'], data['y'], data['z']

# x = x.loc[y.columns, y.columns]
# x = x/x.sum().sum()

# w1 = x.max().max()
# w2 = y.max().max()
# w1 = w1/(w1+w2)
# w2 = 1-w1

# z = (x**w1)*(y**w2)
# z = z/z.sum().sum()
z
    
# get_best_alleles(np.log(z))


PyreadrError: File b'results/hla/imputation/QUILT_HLA_result_method/IDT0606/A/test.RData' does not exist!

In [3]:
data = pyreadr.read_r('results/hla/imputation/QUILT_HLA_result_method/IDT0606/A/test.RData')
x,y = data['x'], data['y']

z = (x**x.max().max())*(y**y.max().max())
z = z/z.sum().sum()
    
get_best_alleles(np.log(z))

,allele1,allele2,prob,cum
0,A*01:01,A*36:01,0.280366,0.280366
1,A*01:01,A*01:01,0.274823,0.555189
2,A*01:01,A*01:02,0.105761,0.660950
3,A*01:01,A*30:01,0.059779,0.720729
4,A*01:02,A*36:01,0.049311,0.770040
...,...,...,...,...
129,A*01:01,A*31:12,0.000260,0.989102
130,A*01:01,A*31:16,0.000260,0.989362
131,A*01:02,A*03:02,0.000249,0.989612
132,A*24:20,A*36:01,0.000235,0.989846


In [46]:
mate[['A*01:01:01:11', 'A*36:01']]

,A*01:01:01:11,A*36:01
A00711:748:HHFNVDSX7:4:1230:18710:27226,-8.515172,-8.515172
A00711:748:HHFNVDSX7:4:1646:14940:22952,-1.251082,-1.251082
A00711:753:HHCG3DSX7:4:1125:31448:8422,-0.156067,-0.156067
A00711:753:HHCG3DSX7:4:1237:17517:18959,-0.132365,-0.132365
A00711:753:HHCG3DSX7:4:1257:9796:34460,-0.640918,-0.640918
A00711:753:HHCG3DSX7:4:1471:17219:28119,-0.629067,-0.629067
A00711:753:HHCG3DSX7:4:1474:4887:33317,-0.312134,-0.312134
A00711:753:HHCG3DSX7:4:1620:3233:22091,-8.150836,-8.150836
A00711:753:HHCG3DSX7:4:2102:22101:20697,-0.861039,-0.861039
A00711:753:HHCG3DSX7:4:2220:20455:13369,-37.176296,-37.176296


# WFA piece

In [55]:
db = pd.read_csv('../recyclable_files/hla_reference_files/v3390_oneKG_only/A.ssv', sep = ' ')
ix = 21
seq = r1.loc[ix,'sequence']
bq = r1.loc[ix,'base_quality']

# seq = reverse_complement(seq)
# bq = bq[::-1]

# a = 'DRB1*13:02:01:01'
# a = 'DRB1*03:02:01'
a = 'A*01:01:01:11'
# a = 'A*36:01'

refseq = (''.join(db[a].tolist())).replace('.', '').lstrip('*').rstrip('*')
ref = pywfa.WavefrontAligner(refseq)
result = ref(seq)
result = ref(seq,clip_cigar=True, min_aligned_bases_left=10, min_aligned_bases_right=10)
cigars_lst = result.cigartuples
refstart, refend, readstart, readend = result.pattern_start, result.pattern_end, result.text_start, result.text_end
newrefseq, newseq, newbq = recode_sequences(refseq, seq, bq, cigars_lst, refstart, refend, readstart, readend, rl = 151)
score = calculate_score_per_alignment(newrefseq, newseq, newbq)
# score = adjust_clipping(score, cigars_lst)
result

    score: -6310
    pattern_start: 1345
    pattern_end: 1496
    text_start: 0
    text_end: 151
    cigartuples: [(0, 151)]
    pattern: CAGGAGCAGAGGGGTCAGGGCGAAGTCCCAGGGCCCCAGGCGTGGCTCTCAGGGTCTCAGGCCCCGAAGGCGGTGTATGGATTGGGGAGTCCCAGCCTTGGGGATTCCCCAACTCCGCAGTTTCTTTTCTCCCTCTCCCAACCTACGTAGGGTCCTTCATCCTGGATACTCACGACGCGGACCCAGTTCTCACTCCCATTGGGTGTCGGGTTTCCAGAGAAGCCAATCAGTGTCGTCGCGGTCGCTGTTCTAAAGTCCGCACGCACCCACCGGGACTCAGATTCTCCCCAGACGCCGAGGATGGCCGTCATGGCGCCCCGAACCCTCCTCCTGCTACTCTCGGGGGCCCTGGCCCTGACCCAGACCTGGGCGGGTGAGTGCGGGGTCGGGAGGGAAACCGCCTCTGCGGGGAGAAGCAAGGGGCCCTCCTGGCGGGGGCGCAGGACCGGGGGAGCCGCGCCGGGAGGAGGGTCGGGCAGGTCTCAGCCACTGCTCGCCCCCAGGCTCCCACTCCATGAGGTATTTCTTCACATCCGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAGAAGATGGAGCCGCGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAGTATTGGGACCAGGAGACACGGAATATGAAGGCCCACTCACAGACTGACCGAGCGAACCTGGGGACCCTGCGCGGCTACTACAACCAGAGCGAGGACGGTGAGTGACCCCGGCCCGGGGCGCAGGTCACGACCCCTCATCCCCCACGGACGGGCCAGGTCGCCCACAGTCTCCGGGTCCGAGATC

In [68]:
newseq[-12:]

'AGTTCTCTTTGG'

In [69]:
newrefseq[-12:]

'GGTTCTCCTTGG'